In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from transformers import BertTokenizer, BertForQuestionAnswering
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

question, text = "Who was Jim Henson?", "Jim Henson  was a nice puppet"
inputs = tokenizer(question, text, return_tensors='pt')
start_positions = torch.tensor([1])
end_positions = torch.tensor([3])

outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits

from transformers import ElectraTokenizer
#tokenizer = ElectraTokenizer.from_pretrained('https://huggingface.co/bert-base-uncased')
tokenizer = ElectraTokenizer.from_pretrained('hfl/chinese-electra-180g-base-discriminator')
inputs = tokenizer("你来自中国吗", return_tensors="pt")
outputs = model(**inputs)
last_hidden_states = outputs.last_hidden_state
last_hidden_states

all_tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)])

In [ ]:
#answer

a,b = "Who was Jim Henson?", "Jim Henson  is not a nice puppet"
inputs = tokenizer(a, b, return_tensors='pt')
start_positions = torch.tensor([1])
end_positions = torch.tensor([15])
outputs = model(**inputs, start_positions=start_positions, end_positions=end_positions)
loss = outputs.loss
start_scores = outputs.start_logits
end_scores = outputs.end_logits

from transformers import BertTokenizer, TFBertForQuestionAnswering
import tensorflow as tf

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertForQuestionAnswering.from_pretrained('bert-base-cased')

question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
input_dict = tokenizer(question, text, return_tensors='tf')
outputs = model(**input_dict)
start_logits = outputs.start_logits
end_logits = outputs.end_logits

all_tokens = tokenizer.convert_ids_to_tokens(input_dict["input_ids"].numpy()[0])
answer = ' '.join(all_tokens[tf.math.argmax(start_logits, 1)[0] : tf.math.argmax(end_logits, 1)[0]+1])

from transformers import ElectraTokenizer, ElectraModel
import torch

tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
model = ElectraModel.from_pretrained('google/electra-small-discriminator')

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

# from github baseline

# import package

In [1]:
#必须加gc，垃圾回收机制
import gc
#import logging
from dataclasses import dataclass
from typing import List, Dict
import json
from tqdm import tqdm
from transformers import PreTrainedTokenizer, BasicTokenizer
from transformers.tokenization_utils import _is_whitespace, _is_punctuation, _is_control
import numpy as np
import torch
from torch.utils.data import Dataset, TensorDataset
#import json
import sys
#import argparse
from transformers import ElectraTokenizer
#from data_process_utils import * 
#No.1 independence package
import gzip
import pickle
import os
from os.path import join
#训练
#import argparse
from transformers import AutoModel, AutoConfig,  AutoTokenizer
#from model import MultiSpanQA
#from data_process_utils import *
#import torch
import random
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import pickle
import gzip
from transformers import AdamW, get_linear_schedule_with_warmup
import logging
import torch.nn as nn
from torch.nn import CrossEntropyLoss, BCELoss
from tqdm import trange, tqdm
import timeit
#import os
#from evaluate_utils import compute_predictions
#from evaluate_2021 import CJRCEvaluator
#from os.path import join

from collections import Counter, OrderedDict



# Data pre pocession utils

In [16]:


YES_TOKEN = "[unused1]"
NO_TOKEN = "[unused2]"


class CAILExample:
    def __init__(self,
                 qas_id: str,
                 question_text: str,
                 context_text: str,
                 answer_texts: List[str],
                 answer_start_indexes: List[int],
                 is_impossible: bool,
                 is_yes_no: bool,
                 is_multi_span: bool,
                 answers: List,
                 case_id: str,
                 case_name: str):
        self.qas_id = qas_id
        self.question_text = question_text
        self.context_text = context_text
        self.answer_texts = answer_texts
        self.answer_start_indexes = answer_start_indexes
        self.is_impossible = is_impossible
        self.is_yes_no = is_yes_no
        self.is_multi_span = is_multi_span
        self.answers = answers
        self.case_id = case_id
        self.case_name = case_name

        self.doc_tokens = []
        self.char_to_word_offset = []

        raw_doc_tokens = customize_tokenizer(context_text, True)
        #转控制与空白与标点符号中文为 空格+字符+空格，+split
        #其他直接字符
        k = 0
        temp_word = ""
        for char in self.context_text:
            #分行处解决，#question 为什么不直接复制raw_doc_tokens，可能要生成char_to_word_offset
            #char_to_word_offset（空白的k-1，非空白的k，k在temp_word.lower() == raw_doc_tokens[k]改变）
            #处理其中的英文字符，数字？
            if _is_whitespace(char):
                self.char_to_word_offset.append(k - 1)
                continue
            else:
                temp_word += char
                self.char_to_word_offset.append(k)
            if temp_word.lower() == raw_doc_tokens[k]:
                self.doc_tokens.append(temp_word)#temp_word是单词
                temp_word = ""
                k += 1
        assert k == len(raw_doc_tokens)

        if answer_texts is not None:  # if for training
            start_positions = []
            end_positions = []

            if not is_impossible and not is_yes_no:
                for i in range(len(answer_texts)):
                    answer_offset = context_text.index(answer_texts[i])
                    answer_length = len(answer_texts[i])
                    start_position = self.char_to_word_offset[answer_offset]#第几个词里，而非第几个字符里
                    end_position = self.char_to_word_offset[answer_offset + answer_length - 1]
                    start_positions.append(start_position)
                    end_positions.append(end_position)
            else:
                start_positions.append(-1)
                end_positions.append(-1)
            self.start_positions = start_positions
            self.end_positions = end_positions
    #认为是抽取形的答案，因为结束index=开始+答案长度
    def __repr__(self):
        string = ""
        for key, value in self.__dict__.items():
            string += f"{key}: {value}\n"
        return f"<{self.__class__}>"


@dataclass
class CAILFeature:
    input_ids: List[int]
    attention_mask: List[int]
    token_type_ids: List[int]
    cls_index: int
    p_mask: List
    example_index: int
    unique_id: int
    paragraph_len: int
    token_is_max_context: object
    tokens: List
    token_to_orig_map: Dict
    start_positions: List[int]
    end_positions: List[int]
    is_impossible: bool


@dataclass
class CAILResult:
    unique_id: int
    start_logits: torch.Tensor
    end_logits: torch.Tensor


def read_examples(file: str, is_training: bool) -> List[CAILExample]:
    example_list = []
    with open(file, "r", encoding="utf-8") as file:
        original_data = json.load(file)["data"]

    for entry in tqdm(original_data, disable=True):
        case_id = entry["caseid"]
        for paragraph in entry["paragraphs"]:
            context = paragraph["context"]
            case_name = paragraph["casename"]
            for qa in paragraph["qas"]:
                question = qa["question"]
                qas_id = qa["id"]
                answer_texts = None
                answer_starts = None
                is_impossible = None
                is_yes_no = None
                is_multi_span = None
                all_answers = None
                if is_training:
                    all_answers = qa["answers"]
                    if len(all_answers) == 0:
                        answer = []
                    else:
                        answer = all_answers[0]
                    # a little difference between 19 and 21 data.
                    #19年data里是内容，21年是[内容]
                    #因为19年每个问句中只有一个提问，21年有如此提问：什么时候对a案件做出了什么样的宣判？
                    if type(answer) == dict:
                        answer = [answer]

                    if len(answer) == 0:  # NO Answer
                        answer_texts = [""]
                        answer_starts = [-1]
                    else:
                        answer_texts = []
                        answer_starts = []
                        for a in answer:
                            answer_texts.append(a["text"])
                            answer_starts.append(a["answer_start"])
                    # Judge YES or NO
                    if len(answer_texts) == 1 and answer_starts[0] == -1 and (answer_texts[0] == "YES" or answer_texts[0] == "NO"):
                        is_yes_no = True
                    else:
                        is_yes_no = False
                    # Judge Multi Span
                    if len(answer_texts) > 1:
                        is_multi_span = True
                    else:
                        is_multi_span = False
                    # Judge No Answer
                    if len(answer_texts) == 1 and answer_texts[0] == "":
                        is_impossible = True
                    else:
                        is_impossible = False

                example = CAILExample(
                    qas_id=qas_id,
                    question_text=question,
                    context_text=context,
                    answer_texts=answer_texts,
                    answer_start_indexes=answer_starts,
                    is_impossible=is_impossible,
                    is_yes_no=is_yes_no,
                    is_multi_span=is_multi_span,
                    answers=all_answers,
                    case_id=case_id,
                    case_name=case_name
                )
                # Discard possible bad example
                if is_training and example.answer_start_indexes[0] >= 0:
                    for i in range(len(example.answer_texts)):
                        actual_text = "".join(example.doc_tokens[example.start_positions[i]: (example.end_positions[i] + 1)])
                        cleaned_answer_text = "".join(whitespace_tokenize(example.answer_texts[i]))#去除空格
                        if actual_text.find(cleaned_answer_text) == -1:
                            print(f"Could not find answer: {actual_text} vs. {cleaned_answer_text}")
                            continue
                example_list.append(example)
    return example_list


def convert_examples_to_features(example_list: List[CAILExample], tokenizer: PreTrainedTokenizer, args,
                                 is_training: bool) -> List[CAILFeature]:
    # Validate there are no duplicate ids in example_list
    qas_id_set = set()
    for example in example_list:
        if example.qas_id in qas_id_set:
            raise Exception("Duplicate qas_id!")
        else:
            qas_id_set.add(example.qas_id)

    feature_list = []
    unique_id = 0
    example_index = 0
    i = 0
    for example in tqdm(example_list, disable=True):
        i += 1
        if i % 1000 == 0:
            print(i)
        current_example_features = convert_single_example_to_features(example, tokenizer, args.max_seq_length,
                                                                      args.max_query_length, args.doc_stride, is_training)
        for feature in current_example_features:
            feature.example_index = example_index#对特征所属的example编码
            feature.unique_id = unique_id
            unique_id += 1
        example_index += 1
        #unique_id 和example_index都是编码
        feature_list.extend(current_example_features)

    return feature_list


def convert_single_example_to_features(example: CAILExample, tokenizer: PreTrainedTokenizer,
                                       max_seq_length, max_query_length, doc_stride, is_training) -> List[CAILFeature]:
    """
    Transfer original text to sequence which can be accepted by ELECTRA
    Format: [CLS] YES_TOKEN NO_TOKEN question [SEP] context [SEP]
    """
    features = []
    tok_to_orig_index = []
    orig_to_tok_index = []
    all_doc_tokens = []
    for (i, token) in enumerate(example.doc_tokens):
        #orig_to_tok_index，token到
        orig_to_tok_index.append(len(all_doc_tokens))
        ##if i==1:
        #    print('info about example token {},{}'.format(type(token),token))
        #token是单个汉字
        sub_tokens = tokenizer.tokenize(token)
        if i==2:
            print('info about example token {},{}'.format(sub_tokens,token))
            #好家伙，几乎不进行编码的，只是str变成了list？
        for sub_token in sub_tokens:
            tok_to_orig_index.append(i)
            all_doc_tokens.append(sub_token)
    #all_doc_token是保存的tokenizer后的token

    if is_training:
        if example.is_impossible or example.answer_start_indexes[0] == -1:
            start_positions = [-1]
            end_positions = [-1]
        else:
            start_positions = []
            end_positions = []
            for i in range(len(example.start_positions)):
                start_position = orig_to_tok_index[example.start_positions[i]]
                if example.end_positions[i] < len(example.doc_tokens) - 1:
                    end_position = orig_to_tok_index[example.end_positions[i] + 1] - 1
                else:
                    end_position = len(all_doc_tokens) - 1
                (start_position, end_position) = _improve_answer_span(
                    all_doc_tokens, start_position, end_position, tokenizer, example.answer_texts[i]
                )
                start_positions.append(start_position)#token级别的
                end_positions.append(end_position)
    else:
        start_positions = None
        end_positions = None

    query_tokens = tokenizer.tokenize(example.question_text)
    query_tokens = [YES_TOKEN, NO_TOKEN] + query_tokens
    truncated_query = tokenizer.encode(query_tokens, add_special_tokens=False, max_length=max_query_length, truncation=True)
    #什么情况
    #Returns the number of added tokens when encoding a sequence with special tokens.
    sequence_pair_added_tokens = tokenizer.num_special_tokens_to_add(pair=True)
    assert sequence_pair_added_tokens == 3
    #pair：Whether the number of added tokens should be computed in the case of a sequence pair or a single sequence.


    added_tokens_num_before_second_sequence = tokenizer.num_special_tokens_to_add(pair=False)
    assert added_tokens_num_before_second_sequence == 2
    #pair true  有第二个句子
    #pair false 第二个句子是None
    #这两句assert有什么意义？，难道是测试num_special_tokens_to_add和build函数
    span_doc_tokens = all_doc_tokens
    spans = []
    print('The length of all_doc_tokens is {}'.format(len(all_doc_tokens)))
    order_while=0
    while len(spans) * doc_stride < len(all_doc_tokens):
#知识点：encode_plus:This method is deprecated, __call__ should be used instead.
        order_while+=1
        encoded_dict = tokenizer.encode_plus(
            truncated_query,
            span_doc_tokens,#？ 分别将问题与其他文本编码？
            max_length=max_seq_length,
            return_overflowing_tokens=True,
            padding="max_length",
            stride=max_seq_length - doc_stride - len(truncated_query) - sequence_pair_added_tokens,
            truncation="only_second",
            return_token_type_ids=True
        )
        #print(encoded_dict.keys())
        print('The length of encoded_dict is{}'.format(len(encoded_dict['input_ids'])))
        paragraph_len = min(
            len(all_doc_tokens) - len(spans) * doc_stride,
            max_seq_length - len(truncated_query) - sequence_pair_added_tokens,
        )

        if tokenizer.pad_token_id in encoded_dict["input_ids"]:
            non_padded_ids = encoded_dict["input_ids"][: encoded_dict["input_ids"].index(tokenizer.pad_token_id)]
        else:
            non_padded_ids = encoded_dict["input_ids"]
        tokens = tokenizer.convert_ids_to_tokens(non_padded_ids)

        token_to_orig_map = {}
        token_to_orig_map[0] = -1
        token_to_orig_map[1] = -1
        token_to_orig_map[2] = -1

        token_is_max_context = {0: True, 1: True, 2: True}
        for i in range(paragraph_len):
            index = len(truncated_query) + added_tokens_num_before_second_sequence + i
            token_to_orig_map[index] = tok_to_orig_index[len(spans) * doc_stride + i]
    #分类编码
        encoded_dict["paragraph_len"] = paragraph_len
        encoded_dict["tokens"] = tokens
        encoded_dict["token_to_orig_map"] = token_to_orig_map
        encoded_dict["truncated_query_with_special_tokens_length"] = len(truncated_query) + added_tokens_num_before_second_sequence
        encoded_dict["token_is_max_context"] = token_is_max_context
        encoded_dict["start"] = len(spans) * doc_stride
        encoded_dict["length"] = paragraph_len

        encoded_dict["token_type_ids"][1] = 1
        encoded_dict["token_type_ids"][2] = 1

        spans.append(encoded_dict)
        #岂不是全都break？
        if "overflowing_tokens" not in encoded_dict or len(encoded_dict["overflowing_tokens"]) == 0:
            #加入测试,每次这里都会运行到
            #**overflowing_tokens** -- List of overflowing tokens sequences (when a :obj:`max_length`
            #is specified and  :obj:`return_overflowing_tokens=True`).
            #print('overflowing_tokens in encoded result in breaking')
            break
        else:
            #print('PositionNo.1 run')
            
            span_doc_tokens = encoded_dict["overflowing_tokens"]
            print('The length of span_doc{} is {}'.format(order_while,len(span_doc_tokens)))
            

    for doc_span_index in range(len(spans)):
        for j in range(spans[doc_span_index]["paragraph_len"]):
            is_max_context = _new_check_is_max_context(spans, doc_span_index, doc_span_index * doc_stride + j)
            index = spans[doc_span_index]["truncated_query_with_special_tokens_length"] + j
            spans[doc_span_index]["token_is_max_context"][index] = is_max_context

    for span in spans:
        cls_index = span["input_ids"].index(tokenizer.cls_token_id)

        # p_mask: mask with 1 for token than cannot be in the answer (0 for token which can be in an answer)
        p_mask = np.array(span["token_type_ids"])#0是前一段，即问题，1是后一段即文本
        p_mask = np.minimum(p_mask, 1)
        p_mask = 1 - p_mask
        p_mask[np.where(np.array(span["input_ids"]) == tokenizer.sep_token_id)[0]] = 1
        p_mask[cls_index] = 0
        p_mask[1] = 0
        p_mask[2] = 0

        current_start_positions = None
        current_end_positions = None
        span_is_impossible = None
        if is_training:
            current_start_positions = [0 for i in range(max_seq_length)]
            current_end_positions = [0 for i in range(max_seq_length)]
            doc_start = span["start"]
            doc_end = span["start"] + span["length"] - 1
            doc_offset = len(truncated_query) + added_tokens_num_before_second_sequence#num of added tokens+2
            #context的位置
            for i in range(len(start_positions)):
                start_position = start_positions[i]#token position
                end_position = end_positions[i]
                if start_position >= doc_start and end_position <= doc_end:#都转到原始文本的token级进行比较
                    span_is_impossible = False#span内有答案
                    current_start_positions[start_position - doc_start + doc_offset] = 1
                    current_end_positions[end_position - doc_start + doc_offset] = 1
                    #用以区分跨span？
#19年有许多YES，No问题，以answer_start=-1标记
#19年的big train data 含有is_impossible='True'的情况，代表空答案
            if example.is_yes_no:
                assert len(example.answer_start_indexes) == 1
                assert 1 not in current_start_positions and 1 not in current_end_positions
                #知识点:检查数据可以使用assert，表明is_yes_no问题答案不是来自文本的抽取答案
                #没有进answer的划分且answer_start_indexes只有一个内容
                if example.answer_texts[0] == "YES" and example.answer_start_indexes[0] == -1:
                    current_start_positions[1] = 1
                    current_end_positions[1] = 1
                elif example.answer_texts[0] == "NO" and example.answer_start_indexes[0] == -1:
                    current_start_positions[2] = 1
                    current_end_positions[2] = 1
                else:
                    raise Exception("example构造出错,请检查")
                span_is_impossible = False

            if 1 not in current_start_positions:  # Current Feature does not contain answer span
                span_is_impossible = True
                current_start_positions[cls_index] = 1#把答案位置调节到cls在的index
                current_end_positions[cls_index] = 1#把答案位置调节到cls在的index
            assert span_is_impossible is not None
            #注意如何对跨span答案和yes_no问题区分的
        features.append(
            CAILFeature(
                input_ids=span["input_ids"],
                attention_mask=span["attention_mask"],
                token_type_ids=span["token_type_ids"],
                cls_index=cls_index,
                p_mask=p_mask.tolist(),
                example_index=0,#在外部进行编码
                unique_id=0,#在外部进行编码
                paragraph_len=span["paragraph_len"],
                token_is_max_context=span["token_is_max_context"],
                tokens=span["tokens"],
                token_to_orig_map=span["token_to_orig_map"],
                start_positions=current_start_positions,
                end_positions=current_end_positions,
                is_impossible=span_is_impossible
            )
        )#features里面是数个span，每个span由question，context和答案组成
    return features


def convert_features_to_dataset(features: List[CAILFeature], is_training: bool) -> Dataset:
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_masks = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_cls_index = torch.tensor([f.cls_index for f in features], dtype=torch.long)
    all_p_mask = torch.tensor([f.p_mask for f in features], dtype=torch.float)
    all_example_indexes = torch.tensor([f.example_index for f in features], dtype=torch.long)
    all_feature_indexes = torch.arange(all_input_ids.size(0), dtype=torch.long)
    if is_training:
        all_is_impossible = torch.tensor([f.is_impossible for f in features], dtype=torch.float)
        all_start_labels = torch.tensor([f.start_positions for f in features], dtype=torch.float)
        all_end_labels = torch.tensor([f.end_positions for f in features], dtype=torch.float)
        dataset = TensorDataset(
            all_input_ids,
            all_attention_masks,
            all_token_type_ids,
            all_start_labels,
            all_end_labels,
            all_cls_index,
            all_p_mask,
            all_is_impossible,
            all_example_indexes,
            all_feature_indexes
        )#dataset是所有的数据组成的数据集
    else:
        dataset = TensorDataset(
            all_input_ids,
            all_attention_masks,
            all_token_type_ids,
            all_cls_index,
            all_p_mask,
            all_example_indexes,
            all_feature_indexes
        )
    return dataset


def _is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False


def _new_check_is_max_context(doc_spans, cur_span_index, position):
    """
    Check if this is the 'max context' doc span for the token.
    """
    # if len(doc_spans) == 1:
    # return True
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span["start"] + doc_span["length"] - 1
        if position < doc_span["start"]:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span["start"]
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span["length"]#和length有关和position的划分有关
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index#所有的span才返回一个true


def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer, orig_answer_text):
    """
    Returns tokenized answer spans that better match the annotated answer.
    没太明白,原本答案文本和input_start到input_end之间对应的原始文本不应该本就对应吗,为什么还可能找出一个对应的子区间?...
    """
    tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))

    for new_start in range(input_start, input_end + 1):
        for new_end in range(input_end, new_start - 1, -1):
            text_span = " ".join(doc_tokens[new_start : (new_end + 1)])
            if text_span == tok_answer_text:
                return (new_start, new_end)

    return (input_start, input_end)


def customize_tokenizer(text: str, do_lower_case=True) -> List[str]:
    temp_x = ""
    for char in text:
        if _is_chinese_char(ord(char)) or _is_punctuation(char) or _is_whitespace(char) or _is_control(char):
            temp_x += " " + char + " "
        else:
            temp_x += char
    if do_lower_case:
        temp_x = temp_x.lower()
    return temp_x.split()


def _is_chinese_char(cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if (
            (cp >= 0x4E00 and cp <= 0x9FFF)
            or (cp >= 0x3400 and cp <= 0x4DBF)  #
            or (cp >= 0x20000 and cp <= 0x2A6DF)  #
            or (cp >= 0x2A700 and cp <= 0x2B73F)  #
            or (cp >= 0x2B740 and cp <= 0x2B81F)  #
            or (cp >= 0x2B820 and cp <= 0x2CEAF)  #
            or (cp >= 0xF900 and cp <= 0xFAFF)
            or (cp >= 0x2F800 and cp <= 0x2FA1F)  #
    ):  #
        return True

    return False


def whitespace_tokenize(text: str):
    if text is None:
        return []
    text = text.strip()
    tokens = text.split()
    return tokens


def write_example_orig_file(examples: List[CAILExample], file: str):
    """
    convert examples to original json file
    """
    data_list = []
    for example in examples:
        data = {
            "paragraphs": [
                {
                    "context": example.context_text,
                    "casename": example.case_name,
                    "qas": [
                        {
                            "question": example.question_text,
                            "answers": example.answers,
                            "id": example.qas_id,
                            "is_impossible": "true" if example.is_impossible else "false",
                        }
                    ]
                }
            ],
            "caseid": example.case_id
        }
        data_list.append(data)
    final_data = {
        "data": data_list,
        "version": "1.0"
    }
    with open(file, mode="w", encoding="utf-8") as file:
        file.write(json.dumps(final_data, ensure_ascii=False))

#test For Data
with open('../input/forcail2021/ydlj_cjrc3.0_small_train.json', "r", encoding="utf-8") as file:
        original_data = json.load(file)

type(original_data['data'])

In [ ]:
tokenized_example={'input_ids': [[101, 2129, 2116, 5222, 2515, 1996, 10289, 8214, 2273, 1005, 1055, 3455, 2136, 2031, 1029, 102, 1996, 2273, 1005, 1055, 3455, 2136, 2038, 2058, 1015, 1010, 5174, 5222, 1010, 2028, 1997, 2069, 2260, 2816, 2040, 2031, 2584, 2008, 2928, 1010, 1998, 2031, 2596, 1999, 2654, 5803, 8504, 1012, 2280, 2447, 5899, 12385, 4324, 1996, 2501, 2005, 2087, 2685, 3195, 1999, 1037, 2309, 2208, 1997, 1996, 2977, 2007, 6079, 1012, 2348, 1996, 2136, 2038, 2196, 2180, 1996, 5803, 2977, 1010, 2027, 2020, 2315, 2011, 1996, 16254, 2015, 5188, 3192, 2004, 2120, 3966, 3807, 1012, 1996, 2136, 2038, 23339, 1037, 2193, 1997, 6314, 2015, 1997, 2193, 2028, 4396, 2780, 1010, 1996, 2087, 3862, 1997, 2029, 2001, 4566, 12389, 1005, 1055, 2501, 6070, 1011, 2208, 3045, 9039, 1999, 3326, 1012, 1996, 2136, 2038, 7854, 2019, 3176, 2809, 2193, 1011, 2028, 2780, 1010, 1998, 2216, 3157, 5222, 4635, 2117, 1010, 2000, 12389, 1005, 1055, 2184, 1010, 2035, 1011, 2051, 1999, 5222, 2114, 1996, 2327, 2136, 1012, 1996, 2136, 3248, 1999, 4397, 10601, 26429, 10531, 1006, 2306, 1996, 9493, 1052, 1012, 11830, 2415, 1007, 1010, 2029, 11882, 2005, 1996, 2927, 1997, 1996, 2268, 1516, 2230, 2161, 1012, 1996, 2136, 2003, 8868, 2011, 3505, 7987, 3240, 1010, 2040, 1010, 2004, 1997, 1996, 2297, 1516, 2321, 2161, 1010, 2010, 16249, 2012, 10289, 8214, 1010, 2038, 4719, 1037, 29327, 1011, 13913, 2501, 1012, 1999, 2268, 2027, 2020, 4778, 2000, 1996, 9152, 2102, 1010, 2073, 2027, 3935, 2000, 1996, 8565, 2021, 2020, 7854, 2011, 9502, 2110, 2040, 2253, 2006, 1998, 3786, 23950, 1999, 1996, 2528, 1012, 1996, 2230, 1516, 2340, 2136, 5531, 2049, 3180, 2161, 4396, 2193, 2698, 1999, 1996, 2406, 1010, 2007, 1037, 2501, 1997, 2423, 1516, 1019, 1010, 7987, 3240, 1005, 1055, 3587, 3442, 2322, 1011, 2663, 2161, 1010, 1998, 1037, 2117, 1011, 2173, 3926, 1999, 1996, 2502, 2264, 1012, 2076, 1996, 2297, 1011, 2321, 2161, 1010, 1996, 2136, 2253, 3590, 1011, 1020, 1998, 2180, 1996, 16222, 3034, 2977, 1010, 2101, 10787, 2000, 1996, 7069, 1022, 1010, 2073, 1996, 3554, 3493, 2439, 2006, 1037, 4771, 12610, 2121, 1011, 3786, 2121, 2114, 2059, 15188, 5612, 1012, 2419, 2011, 6452, 4433, 11214, 15333, 6862, 3946, 1998, 6986, 9530, 2532, 18533, 2239, 1010, 1996, 3554, 3493, 3786, 1996, 9523, 2120, 3410, 3804, 2630, 13664, 3807, 2076, 1996, 2161, 1012, 1996, 3590, 5222, 2020, 102], [101, 2129, 2116, 5222, 2515, 1996, 10289, 8214, 2273, 1005, 1055, 3455, 2136, 2031, 1029, 102, 2528, 1012, 1996, 2230, 1516, 2340, 2136, 5531, 2049, 3180, 2161, 4396, 2193, 2698, 1999, 1996, 2406, 1010, 2007, 1037, 2501, 1997, 2423, 1516, 1019, 1010, 7987, 3240, 1005, 1055, 3587, 3442, 2322, 1011, 2663, 2161, 1010, 1998, 1037, 2117, 1011, 2173, 3926, 1999, 1996, 2502, 2264, 1012, 2076, 1996, 2297, 1011, 2321, 2161, 1010, 1996, 2136, 2253, 3590, 1011, 1020, 1998, 2180, 1996, 16222, 3034, 2977, 1010, 2101, 10787, 2000, 1996, 7069, 1022, 1010, 2073, 1996, 3554, 3493, 2439, 2006, 1037, 4771, 12610, 2121, 1011, 3786, 2121, 2114, 2059, 15188, 5612, 1012, 2419, 2011, 6452, 4433, 11214, 15333, 6862, 3946, 1998, 6986, 9530, 2532, 18533, 2239, 1010, 1996, 3554, 3493, 3786, 1996, 9523, 2120, 3410, 3804, 2630, 13664, 3807, 2076, 1996, 2161, 1012, 1996, 3590, 5222, 2020, 1996, 2087, 2011, 1996, 3554, 3493, 2136, 2144, 5316, 1011, 5641, 1012, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'overflow_to_sample_mapping': [0, 0]}
tokenized_example.keys()

In [ ]:
dict_keys(['overflowing_tokens', 'num_truncated_tokens', 'input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])

In [12]:
gc.collect()

# Data classifacation and some evaluation

In [6]:
OPTS = None

TYPE_YES = "Yes"
TYPE_NO = "No"
TYPE_NO_ANSWER = "No Answer"
TYPE_SINGLE_SPAN = "Single Span"
TYPE_MULTI_SPAN = "Multi Span"

In [6]:

#from data_process_utils import CAILExample, read_examples





class CJRCEvaluator:
    def __init__(self, gold_file):
        examples = read_examples(gold_file, True)
        id_to_example = {}
        for example in examples:
            id_to_example[example.qas_id] = example
        self.id_to_example = id_to_example
        self.gold_data = CJRCEvaluator.gold_answers_to_dict(gold_file)

    @staticmethod
    def gold_answers_to_dict(gold_file):
        dataset = json.load(open(gold_file, mode="r", encoding="utf-8"))
        gold_dict = {}
        # id_to_domain = {}
        for story in dataset['data']:
            qas = story["paragraphs"][0]["qas"]
            for qa in qas:
                qid = qa['id']
                gold_answers = []
                answers = qa["answers"]
                if len(answers) == 0:
                    gold_answers = ['']
                else:
                    for answer in qa["answers"]:
                        if type(answer) == dict:
                            gold_answers.append(answer["text"])
                        elif type(answer) == list:
                            gold_answers.append("".join([a["text"] for a in answer]))
                if qid in gold_dict:
                    sys.stderr.write("Gold file has duplicate stories: {}".format(qid))
                gold_dict[qid] = gold_answers
        return gold_dict

    @staticmethod
    def preds_to_dict(pred_file):
        preds = json.load(open(pred_file, mode="r", encoding="utf-8"))
        pred_dict = {}
        for pred in preds:
            pred_dict[pred['id']] = "".join(pred['answer'])
        return pred_dict

    @staticmethod
    def normalize_answer(s):
        """Lower text and remove punctuation, storys and extra whitespace."""

        def remove_punc(text):
            return "".join(ch for ch in text if ch.isdigit() or ch.isalpha())

        def lower(text):
            return text.lower()
    
        return remove_punc(lower(s))

    @staticmethod
    def get_tokens(s):
        if not s: return []
        return list(CJRCEvaluator.normalize_answer(s))

    @staticmethod
    def compute_exact(a_gold, a_pred):
        return int(CJRCEvaluator.normalize_answer(a_gold) == CJRCEvaluator.normalize_answer(a_pred))

    @staticmethod
    def compute_f1(a_gold, a_pred):
        gold_toks = CJRCEvaluator.get_tokens(a_gold)
        pred_toks = CJRCEvaluator.get_tokens(a_pred)
        common = Counter(gold_toks) & Counter(pred_toks)
        num_same = sum(common.values())
        if len(gold_toks) == 0 or len(pred_toks) == 0:
            # If either is no-answer, then F1 is 1 if they agree, 0 otherwise
            return int(gold_toks == pred_toks)
        if num_same == 0:
            return 0
        precision = 1.0 * num_same / len(pred_toks)
        recall = 1.0 * num_same / len(gold_toks)
        f1 = (2 * precision * recall) / (precision + recall)
        return f1

    @staticmethod
    def _compute_turn_score(a_gold_list, a_pred):
        f1_sum = 0.0
        em_sum = 0.0
        if len(a_gold_list) > 1:
            for i in range(len(a_gold_list)):
                # exclude the current answer
                gold_answers = a_gold_list[0:i] + a_gold_list[i + 1:]
                em_sum += max(CJRCEvaluator.compute_exact(a, a_pred) for a in gold_answers)
                f1_sum += max(CJRCEvaluator.compute_f1(a, a_pred) for a in gold_answers)
        else:
            em_sum += max(CJRCEvaluator.compute_exact(a, a_pred) for a in a_gold_list)
            f1_sum += max(CJRCEvaluator.compute_f1(a, a_pred) for a in a_gold_list)
        if f1_sum != 1:
            a = 1 + 1
        return {'em': em_sum / max(1, len(a_gold_list)), 'f1': f1_sum / max(1, len(a_gold_list))}

    @staticmethod
    def _compute_turn_score1(a_gold_list, a_pred):
        f1 = 0.0
        em = 0.0
        if len(a_gold_list) > 1:
            for i in range(len(a_gold_list)):
                # exclude the current answer
                em = max(em, CJRCEvaluator.compute_exact(a_gold_list[i], a_pred))
                f1 = max(f1, CJRCEvaluator.compute_f1(a_gold_list[i], a_pred))
        else:
            em = max(CJRCEvaluator.compute_exact(a, a_pred) for a in a_gold_list)
            f1 = max(CJRCEvaluator.compute_f1(a, a_pred) for a in a_gold_list)
        if em != 1 or f1 != 1:
            a = 1 + 1
        return {'em': em, 'f1': f1}

    def compute_turn_score(self, qid, a_pred):
        ''' This is the function what you are probably looking for. a_pred is the answer string your model predicted. '''
        a_gold_list = self.gold_data[qid]
        return CJRCEvaluator._compute_turn_score(a_gold_list, a_pred)

    def get_raw_scores(self, pred_data):
        ''''Returns a dict with score'''
        exact_scores = {}
        f1_scores = {}
        for qid in self.gold_data:
            if qid not in pred_data:
                sys.stderr.write('Missing prediction for {}\n'.format(qid))
                continue
            a_pred = pred_data[qid]
            scores = self.compute_turn_score(qid, a_pred)
            # Take max over all gold answers
            exact_scores[qid] = scores['em']
            f1_scores[qid] = scores['f1']
        return exact_scores, f1_scores

    def get_raw_scores_human(self):
        '''
        Returns a dict with score
        '''
        exact_scores = {}
        f1_scores = {}
        for qid in self.gold_data:
            f1_sum = 0.0
            em_sum = 0.0
            if len(self.gold_data[qid]) > 1:
                for i in range(len(self.gold_data[qid])):
                    # exclude the current answer
                    gold_answers = self.gold_data[qid][0:i] + self.gold_data[qid][i + 1:]
                    em_sum += max(CJRCEvaluator.compute_exact(a, self.gold_data[qid][i]) for a in gold_answers)
                    f1_sum += max(CJRCEvaluator.compute_f1(a, self.gold_data[qid][i]) for a in gold_answers)
            else:
                exit("Gold answers should be multiple: {}={}".format(qid, self.gold_data[qid]))
            exact_scores[qid] = em_sum / len(self.gold_data[qid])
            f1_scores[qid] = f1_sum / len(self.gold_data[qid])
        return exact_scores, f1_scores

    def human_performance(self):
        exact_scores, f1_scores = self.get_raw_scores_human()
        return self.get_total_scores(exact_scores, f1_scores)

    def model_performance(self, pred_data):
        exact_scores, f1_scores = self.get_raw_scores(pred_data)
        return self.get_total_scores(exact_scores, f1_scores)
        # return self.get_qa_types_score(exact_scores, f1_scores)

    def get_total_scores(self, exact_scores, f1_scores):
        em_total, f1_total, turn_count = 0, 0, 0
        scores = {}
        for qid in self.gold_data:
            em_total += exact_scores.get(qid, 0)
            f1_total += f1_scores.get(qid, 0)
            turn_count += 1
        scores["overall"] = {'em': round(em_total / max(1, turn_count) * 100, 1),
                             'f1': round(f1_total / max(1, turn_count) * 100, 1),
                             'qas': turn_count}
        return scores

    def get_qa_types_score(self, exact_scores, f1_scores):
        qa_types = {}
        em_total = 0.0
        f1_total = 0.0
        total_count = 0
        for qid in self.gold_data:
            type = get_example_qa_type(self.id_to_example[qid])
            em_total += exact_scores.get(qid, 0)
            f1_total += f1_scores.get(qid, 0)
            total_count += 1
            if type not in qa_types:
                qa_types[type] = {}
                qa_types[type]["em_total"] = exact_scores.get(qid, 0)
                qa_types[type]['f1_total'] = f1_scores.get(qid, 0)
                qa_types[type]['qa_count'] = 1
            else:
                qa_types[type]["em_total"] += exact_scores.get(qid, 0)
                qa_types[type]['f1_total'] += f1_scores.get(qid, 0)
                qa_types[type]['qa_count'] += 1
        scores = OrderedDict()
        for type in qa_types.keys():
            scores[type] = {
                "em": round(qa_types[type]["em_total"] / max(1, qa_types[type]['qa_count']) * 100, 1),
                "f1": round(qa_types[type]['f1_total'] / max(1, qa_types[type]['qa_count']) * 100, 1),
                "qas": qa_types[type]['qa_count']
            }
        scores["F1"] = round(f1_total / max(1, total_count) * 100, 1)
        return scores




In [3]:
def get_example_qa_type(example: CAILExample):
    if example.is_impossible:
        return TYPE_NO_ANSWER
    if example.is_yes_no:
        if example.answer_texts[0] == "YES":
            return TYPE_YES
        elif example.answer_texts[0] == "NO":
            return TYPE_NO
        else:
            raise Exception()
    if len(example.answer_texts) == 1:
        return TYPE_SINGLE_SPAN
    if len(example.answer_texts) > 1:
        return TYPE_MULTI_SPAN
    raise Exception()

# data generation config and run

In [4]:
class ConfigAboutDataSplit:
    train_21 ='../input/forcail2021/ydlj_cjrc3.0_small_train.json'
    dev_19='../input/forcail2019/dev_ground_truth.json'
    train_19='../input/forcail2019/big_train_data.json'
    train_output = 'train.json'
    dev_output ='dev.json'
args_split=ConfigAboutDataSplit()
#

In [7]:
"""
用于对数据划分，使用19年和21年训练集（8/9）得到新的训练集，使用19年开发集和21年训练集（1/9）得到新的开发集
"""
#from data_process_utils import write_example_orig_file
#from evaluate_2021 import *



def main_data_split(args):
    '''
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--train_21",
        type=str,
        required=True,
        help="The train set of CAIL 2021."
    )
    parser.add_argument(
        "--dev_19",
        type=str,
        required=True,
        help="The dev set of CAIL 2019."
    )
    parser.add_argument(
        "--train_19",
        type=str,
        required=True,
        help="The train set of CAIL 2019."
    )
    parser.add_argument(
        "--train_output",
        type=str,
        default="data/train.json",
        help="Directory to write train set after split."
    )
    parser.add_argument(
        "--dev_output",
        type=str,
        default="data/dev.json",
        help="Directory to write dev set after split."
    )
    args = parser.parse_args()

    logging.basicConfig(
        format="%(asctime)s - %(levelname)s: %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
        level=logging.INFO,
    )
    logging.info("All input parameters:")
    print(json.dumps(vars(args), sort_keys=False, indent=2))
    '''
    # if not os.path.exists(args.output_path):
    #     os.makedirs(args.output_path)

    examples_train_21 = read_examples(args.train_21, is_training=True)
    examples_dev_19 = read_examples(args.dev_19, is_training=True)
    examples_train_19 = read_examples(args.train_19, is_training=True)

    random.shuffle(examples_train_21)
    random.shuffle(examples_dev_19)

    # split CAIL 2021 train-set to 8 : 1
    num_per_qa_dev = int(1.0 * len(examples_train_21) * (1 / 9))
    logging.info(f"num per qa type in dev: {num_per_qa_dev}")

    train_set = []
    dev_set = []

    dev_set.extend(examples_train_21[:num_per_qa_dev])
    train_set.extend(examples_train_21[num_per_qa_dev:])
    train_set.extend(examples_train_19[:12*len(examples_train_21)])

    yes_examples_19 = [e for e in examples_dev_19 if get_example_qa_type(e) == TYPE_YES]
    no_examples_19 = [e for e in examples_dev_19 if get_example_qa_type(e) == TYPE_NO]
    single_span_examples_19 = [e for e in examples_dev_19 if get_example_qa_type(e) == TYPE_SINGLE_SPAN]
    no_answer_examples_19 = [e for e in examples_dev_19 if get_example_qa_type(e) == TYPE_NO_ANSWER]

    dev_set.extend(yes_examples_19[:num_per_qa_dev])
    dev_set.extend(no_examples_19[:num_per_qa_dev])
    dev_set.extend(single_span_examples_19[:num_per_qa_dev])
    dev_set.extend(no_answer_examples_19[:num_per_qa_dev])

    print(f"Train set example num: {len(train_set)}")
    print(f"Test set example num: {len(dev_set)}")

    random.shuffle(train_set)
    random.shuffle(dev_set)

    # re-generate id for example to prevent duplicate id between 19 and 21 data
    for i, example in enumerate(train_set):
        example.qas_id = i
    for i, example in enumerate(dev_set):
        example.qas_id = i

    # write split data back to file
    write_example_orig_file(train_set, args.train_output)
    write_example_orig_file(dev_set, args.dev_output)

    print("Complete!")

main_data_split(args_split)


In [8]:
gc.collect()

# data pocess main

In [8]:

class ConfigAboutDataPocess:
    #input_file='../input/forcail2021/train.json' #2020年的数据集不可以
    #input_file='../input/forcail2021/ydlj_cjrc3.0_small_train.json'
    input_file='./train.json'
    #      --for_training \
    #      --output_prefix train \
    #      --do_lower_case \
    #      --tokenizer_path $electra_path \
    max_seq_length=512
    max_query_length=66
    doc_stride=128
    output_path='./pocessed_data/'
    for_training=True
    #args.max_query_length += 2  # position for token yes and no
    #max_query_length += 2
    output_prefix='train'
    
args_data=ConfigAboutDataPocess()
    

In [17]:
"""
测试使用
"""

def convert_and_write(args, tokenizer: PreTrainedTokenizer, file, examples_fn, features_fn, is_training):
    logging.info(f"Reading examples from :{file} ...")
    example_list = read_examples(file, is_training=is_training)[:50]
    #logging.info(f"Total examples:{len(example_list)}")

    #logging.info(f"Start converting examples to features.")
    feature_list = convert_examples_to_features(example_list, tokenizer, args, is_training)
    #logging.info(f"Total features:{len(feature_list)}")

    #logging.info(f"Converting complete, writing examples and features to file.")
    with gzip.open(join(args_data.output_path, examples_fn), "wb") as file:
        pickle.dump(example_list, file)
    with gzip.open(join(args_data.output_path, features_fn), "wb") as file:
        pickle.dump(feature_list, file)


def main_data_prepocess():
    '''
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--input_file",
        type=str,
        required=True,
        help="The file to be processed."
    )

    parser.add_argument(
        "--for_training",
        action="store_true",
        help="Process for training or not."
    )

    parser.add_argument(
        "--output_prefix",
        type=str,
        required=True,
        help="The prefix of output file's name."
    )

    parser.add_argument(
        "--do_lower_case",
        action="store_true",
        help="Set this flag if you are using an uncased model."
    )

    parser.add_argument(
        "--tokenizer_path",
        type=str,
        required=True,
        help="Path to tokenizer which will be used to tokenize text.(ElectraTokenizer)"
    )

    parser.add_argument(
        "--max_seq_length",
        default=512,
        type=int,
        help="The maximum total input sequence length after WordPiece tokenization. "
             "Longer will be truncated, and shorter will be padded."
    )
    parser.add_argument(
        "--max_query_length",
        default=64,
        type=int,
        help="The maximum number of tokens for the question. Questions longer will be truncated to the length."
    )
    parser.add_argument(
        "--doc_stride",
        default=128,
        type=int,
        help="When splitting up a long document into chunks, how much stride to take between chunks."
    )

    parser.add_argument(
        "--output_path",
        default="./processed_data/",
        type=str,
        help="Output path of the constructed examples and features."
    )

    args = parser.parse_args()
    '''
    #####
    #####重新定义参数

    tokenizer = ElectraTokenizer.from_pretrained('hfl/chinese-electra-180g-base-discriminator')

    if not os.path.exists(args_data.output_path):
        os.makedirs(args_data.output_path)
    #生成train文件用
    convert_and_write(args_data, tokenizer, args_data.input_file, args_data.output_prefix+"_examples.pkl.gz",
                       args_data.output_prefix+"_features.pkl.gz", args_data.for_training)


main_data_prepocess()


In [ ]:
"""
数据处理相关代码
"""

def convert_and_write(args, tokenizer: PreTrainedTokenizer, file, examples_fn, features_fn, is_training):
    logging.info(f"Reading examples from :{file} ...")
    example_list = read_examples(file, is_training=is_training)
    #logging.info(f"Total examples:{len(example_list)}")

    #logging.info(f"Start converting examples to features.")
    feature_list = convert_examples_to_features(example_list, tokenizer, args, is_training)
    #logging.info(f"Total features:{len(feature_list)}")

    #logging.info(f"Converting complete, writing examples and features to file.")
    with gzip.open(join(args_data.output_path, examples_fn), "wb") as file:
        pickle.dump(example_list, file)
    with gzip.open(join(args_data.output_path, features_fn), "wb") as file:
        pickle.dump(feature_list, file)


def main_data_prepocess():
    '''
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--input_file",
        type=str,
        required=True,
        help="The file to be processed."
    )

    parser.add_argument(
        "--for_training",
        action="store_true",
        help="Process for training or not."
    )

    parser.add_argument(
        "--output_prefix",
        type=str,
        required=True,
        help="The prefix of output file's name."
    )

    parser.add_argument(
        "--do_lower_case",
        action="store_true",
        help="Set this flag if you are using an uncased model."
    )

    parser.add_argument(
        "--tokenizer_path",
        type=str,
        required=True,
        help="Path to tokenizer which will be used to tokenize text.(ElectraTokenizer)"
    )

    parser.add_argument(
        "--max_seq_length",
        default=512,
        type=int,
        help="The maximum total input sequence length after WordPiece tokenization. "
             "Longer will be truncated, and shorter will be padded."
    )
    parser.add_argument(
        "--max_query_length",
        default=64,
        type=int,
        help="The maximum number of tokens for the question. Questions longer will be truncated to the length."
    )
    parser.add_argument(
        "--doc_stride",
        default=128,
        type=int,
        help="When splitting up a long document into chunks, how much stride to take between chunks."
    )

    parser.add_argument(
        "--output_path",
        default="./processed_data/",
        type=str,
        help="Output path of the constructed examples and features."
    )

    args = parser.parse_args()
    '''
    #####
    #####重新定义参数

    tokenizer = ElectraTokenizer.from_pretrained('hfl/chinese-electra-180g-base-discriminator')

    if not os.path.exists(args_data.output_path):
        os.makedirs(args_data.output_path)
    #生成train文件用
    convert_and_write(args_data, tokenizer, args_data.input_file, args_data.output_prefix+"_examples.pkl.gz",
                       args_data.output_prefix+"_features.pkl.gz", args_data.for_training)


main_data_prepocess()


In [ ]:
gc.collect()

In [ ]:
try:
    print('文件内容')
    print(os.listdir(args_data.output_path))
except:
    print('error before model')

# model define

In [ ]:


class MultiSpanQA(nn.Module):
    def __init__(self, pretrain_model):
        super(MultiSpanQA, self).__init__()
        self.pretrain_model = pretrain_model
        # represent start logits and end logits respectively
        self.qa_outputs = nn.Linear(pretrain_model.config.hidden_size, 2)

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            start_labels=None,  # size: (batch_size, max_seq_length, 1)
            end_labels=None,
    ):
        outputs = self.pretrain_model(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
        sequence_output = outputs[0]
        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = logits.split(1, dim=-1)
        start_logits = start_logits.squeeze(-1)
        end_logits = end_logits.squeeze(-1)
        outputs = (start_logits, end_logits,) + outputs[2:]
        if start_labels is not None and end_labels is not None:
            loss_fct = BCELoss(reduction="mean")
            start_loss = loss_fct(torch.sigmoid(start_logits), start_labels)
            end_loss = loss_fct(torch.sigmoid(end_logits), end_labels)
            total_loss = (start_loss + end_loss) / 2
            outputs = (total_loss,) + outputs
        return outputs

# dont know inner function yet

In [ ]:
  
import collections
import json
import logging
import math
import re
import string
from transformers import BasicTokenizer
from dataclasses import dataclass
from typing import List
#from data_process_utils import CAILExample


@dataclass
class Prediction:
    """
    用来保存可能的预测结果, feature-result-prediction是一一对应的关系
    """
    feature_index: int
    start_index: int  # 预测的对应Sequence中的开始位置
    end_index: int  # 预测的对应Sequence中的结束位置
    start_logit: float  # 预测的开始得分
    end_logit: float  # 预测的截止得分
    text: str = None  # 对应的在原文中的文本,一开始为None,后面被计算
    orig_start_index: int = None
    orig_end_index: int = None
    final_score: int = None


def compute_predictions(
        all_examples: List[CAILExample],
        all_features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        verbose_logging,
        null_score_diff_threshold,
        tokenizer,
        args,
        multi_span_predict: bool = True
):
    example_index_to_features = collections.defaultdict(list)
    for feature in all_features:
        example_index_to_features[feature.example_index].append(feature)
    unique_id_to_result = {}
    for result in all_results:
        unique_id_to_result[result.unique_id] = result

    all_predictions = []
    all_nbest_predict = collections.OrderedDict()

    for (example_index, example) in enumerate(all_examples):
        features = example_index_to_features[example_index]
        predictions = []

        min_cls_score = 1000000
        min_cls_score_feature_index = None
        min_cls_start_logits = 0
        min_cls_end_logits = 0

        for (feature_index, feature) in enumerate(features):
            result = unique_id_to_result[feature.unique_id]

            start_indexes = _get_best_indexes(result.start_logits, n_best_size)
            end_indexes = _get_best_indexes(result.end_logits, n_best_size)

            feature_null_score = result.start_logits[0] + result.end_logits[0]
            if feature_null_score < min_cls_score:
                min_cls_score = feature_null_score
                min_cls_score_feature_index = feature_index
                min_cls_start_logits = result.start_logits[0]
                min_cls_end_logits = result.end_logits[0]

            for start_index in start_indexes:
                for end_index in end_indexes:
                    # We could hypothetically create invalid predictions, e.g., predict
                    # that the start of the span is in the question. We throw out all
                    # invalid predictions.
                    if start_index >= len(feature.tokens):
                        continue
                    if end_index >= len(feature.tokens):
                        continue
                    if start_index not in feature.token_to_orig_map:
                        continue
                    if end_index not in feature.token_to_orig_map:
                        continue
                    if not feature.token_is_max_context.get(start_index, False):
                        continue
                    if end_index < start_index:
                        continue
                    length = end_index - start_index + 1
                    if length > max_answer_length:
                        continue
                    predictions.append(
                        Prediction(
                            feature_index=feature_index,
                            start_index=start_index,
                            end_index=end_index,
                            start_logit=result.start_logits[start_index],
                            end_logit=result.end_logits[end_index],
                        )
                    )
        predictions.append(
            Prediction(
                feature_index=min_cls_score_feature_index,
                start_index=0,
                end_index=0,
                start_logit=min_cls_start_logits,
                end_logit=min_cls_end_logits,
            )
        )

        predictions = sorted(predictions, key=lambda x: (x.start_logit + x.end_logit), reverse=True)
        seen_predictions = {}
        filtered_predictions = []

        for prediction in predictions:
            if len(filtered_predictions) >= n_best_size:
                break

            feature = features[prediction.feature_index]
            if prediction.start_index == 1:
                final_text = "YES"
            elif prediction.start_index == 2:
                final_text = "NO"
            elif prediction.start_index == 0:
                final_text = ""
            else:  # this is a non-null prediction
                tok_tokens = feature.tokens[prediction.start_index: (prediction.end_index + 1)]
                orig_doc_start = feature.token_to_orig_map[prediction.start_index]
                orig_doc_end = feature.token_to_orig_map[prediction.end_index]
                orig_tokens = example.doc_tokens[orig_doc_start: (orig_doc_end + 1)]
                tok_text = tokenizer.convert_tokens_to_string(tok_tokens)

                # tok_text = " ".join(tok_tokens)
                #
                # # De-tokenize WordPieces that have been split off.
                tok_text = tok_text.replace(" ##", "")
                tok_text = tok_text.replace("##", "")

                # Clean whitespace
                tok_text = tok_text.strip()
                tok_text = " ".join(tok_text.split())
                orig_text = " ".join(orig_tokens)

                final_text = get_final_text(tok_text, orig_text, do_lower_case, verbose_logging)
                prediction.orig_start_index = orig_doc_start
                prediction.orig_end_index = orig_doc_end
            if final_text in seen_predictions:
                continue
            seen_predictions[final_text] = True
            prediction.text = final_text
            filtered_predictions.append(prediction)
        predictions = filtered_predictions
        if "" not in seen_predictions:
            predictions.append(
                Prediction(
                    feature_index=min_cls_score_feature_index,
                    start_index=0,
                    end_index=0,
                    start_logit=min_cls_start_logits,
                    end_logit=min_cls_end_logits,
                )
            )
        assert len(predictions) > 0
        if len(predictions) == 1:
            predictions.append(Prediction(feature_index=-1, start_index=-1, end_index=-1,
                                          start_logit=0.0, end_logit=0.0, text="empty"))

        score_normalization(predictions)
        best_non_null_entry = None
        for p in predictions:
            if best_non_null_entry is None and p.text != "":
                best_non_null_entry = p
                break
        score_diff = min_cls_start_logits + min_cls_end_logits - best_non_null_entry.start_logit - best_non_null_entry.end_logit
        predict_answers = []
        if score_diff > null_score_diff_threshold:
            predict_answers = [""]
        else:
            max_score = best_non_null_entry.final_score
            if best_non_null_entry.start_index > 2:
                span_covered = [0 for i in range(len(example.doc_tokens))]
                for p in predictions:
                    if p.start_index > 2 and p.final_score > (max_score * args.multi_span_threshold) \
                            and 1 not in span_covered[p.orig_start_index: (p.orig_end_index + 1)]:
                        predict_answers.append(p.text)
                        span_covered[p.orig_start_index: (p.orig_end_index + 1)] = [1 for i in range(p.orig_start_index, p.orig_end_index + 1)]
            else:
                predict_answers = [best_non_null_entry.text]
        if not multi_span_predict:
            predict_answers = predict_answers[0:1]
        all_predictions.append({"id": example.qas_id, "answer": predict_answers})
        current_nbest = []
        for (i, prediction) in enumerate(predictions):
            output = collections.OrderedDict()
            output["text"] = prediction.text
            output["start_logit"] = prediction.start_logit
            output["end_logit"] = prediction.end_logit
            current_nbest.append(output)
        all_nbest_predict[example.qas_id] = current_nbest

    with open(output_prediction_file, "w", encoding="utf-8") as writer:
        writer.write(json.dumps(all_predictions, indent=4, ensure_ascii=False))

    with open(output_nbest_file, "w", encoding="utf-8") as writer:
        writer.write(json.dumps(all_nbest_predict, indent=4, ensure_ascii=False))

    return all_predictions


def _get_best_indexes(logits, n_best_size):
    """Get the n-best logits from a list."""
    index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

    best_indexes = []
    for i in range(len(index_and_score)):
        if i >= n_best_size:
            break
        best_indexes.append(index_and_score[i][0])
    return best_indexes


def get_final_text(pred_text, orig_text, do_lower_case, verbose_logging=False):
    """Project the tokenized prediction back to the original text."""
    # When we created the data, we kept track of the alignment between original
    # (whitespace tokenized) tokens and our WordPiece tokenized tokens. So
    # now `orig_text` contains the span of our original text corresponding to the
    # span that we predicted.
    #
    # However, `orig_text` may contain extra characters that we don't want in
    # our prediction.
    #
    # For example, let's say:
    #   pred_text = steve smith
    #   orig_text = Steve Smith's
    #
    # We don't want to return `orig_text` because it contains the extra "'s".
    #
    # We don't want to return `pred_text` because it's already been normalized
    # (the SQuAD eval script also does punctuation stripping/lower casing but
    # our tokenizer does additional normalization like stripping accent
    # characters).
    #
    # What we really want to return is "Steve Smith".
    #
    # Therefore, we have to apply a semi-complicated alignment heuristic between
    # `pred_text` and `orig_text` to get a character-to-character alignment. This
    # can fail in certain cases in which case we just return `orig_text`.

    def _strip_spaces(text):
        ns_chars = []
        ns_to_s_map = collections.OrderedDict()
        for (i, c) in enumerate(text):
            if c == " ":
                continue
            ns_to_s_map[len(ns_chars)] = i
            ns_chars.append(c)
        ns_text = "".join(ns_chars)
        return (ns_text, ns_to_s_map)

    # We first tokenize `orig_text`, strip whitespace from the result
    # and `pred_text`, and check if they are the same length. If they are
    # NOT the same length, the heuristic has failed. If they are the same
    # length, we assume the characters are one-to-one aligned.
    tokenizer = BasicTokenizer(do_lower_case=do_lower_case)

    tok_text = " ".join(tokenizer.tokenize(orig_text))

    start_position = tok_text.find(pred_text)
    if start_position == -1:
        if verbose_logging:
            logging.info("Unable to find text: '%s' in '%s'" % (pred_text, orig_text))
        return orig_text
    end_position = start_position + len(pred_text) - 1

    (orig_ns_text, orig_ns_to_s_map) = _strip_spaces(orig_text)
    (tok_ns_text, tok_ns_to_s_map) = _strip_spaces(tok_text)

    if len(orig_ns_text) != len(tok_ns_text):
        if verbose_logging:
            logging.info("Length not equal after stripping spaces: '%s' vs '%s'", orig_ns_text, tok_ns_text)
        return orig_text

    # We then project the characters in `pred_text` back to `orig_text` using
    # the character-to-character alignment.
    tok_s_to_ns_map = {}
    for (i, tok_index) in tok_ns_to_s_map.items():
        tok_s_to_ns_map[tok_index] = i

    orig_start_position = None
    if start_position in tok_s_to_ns_map:
        ns_start_position = tok_s_to_ns_map[start_position]
        if ns_start_position in orig_ns_to_s_map:
            orig_start_position = orig_ns_to_s_map[ns_start_position]

    if orig_start_position is None:
        if verbose_logging:
            logging.info("Couldn't map start position")
        return orig_text

    orig_end_position = None
    if end_position in tok_s_to_ns_map:
        ns_end_position = tok_s_to_ns_map[end_position]
        if ns_end_position in orig_ns_to_s_map:
            orig_end_position = orig_ns_to_s_map[ns_end_position]

    if orig_end_position is None:
        if verbose_logging:
            logging.info("Couldn't map end position")
        return orig_text

    output_text = orig_text[orig_start_position: (orig_end_position + 1)]
    return output_text


def score_normalization(predictions: List[Prediction]):
    scores = [p.start_logit + p.end_logit for p in predictions]
    max_score = max(scores)
    min_score = min(scores)
    for p in predictions:
        if (max_score - min_score) == 0:
            p.final_score = 0
            continue
        p.final_score = 1.0 * ((p.start_logit + p.end_logit) - min_score) / (max_score - min_score)

# Trainer

# might change

In [ ]:
#配置train
class ConfigAboutTrainModel:
    model_path= 'hfl/chinese-electra-180g-base-discriminator'  #huggingface
    train_data_dir='./pocessed_data'
    eval_data_dir='./pocessed_data'
    ground_truth_file='./dev.json'
    output_dir= './output/'
    device = 'cuda'
    batch_size = 8
    #    --do_lower_case \
    #    --logging_steps 200 \
    save_steps = 1000
    gradient_accumulation_steps = 4
    num_train_epochs = 5
    learning_rate = 7e-5
    max_answer_length = 32
    max_seq_length = 512
    max_query_length = 64
    doc_stride = 128
    multi_span_threshold = 0.8
    seed = 42
    weight_decay=0.0
    adam_epsilon=1e-8
    warmup_steps=0
    max_grad_norm=1.0
    logging_steps=50
    save_steps=2000
    
    
args_train=ConfigAboutTrainModel()

In [ ]:
#t=MultiSpanQA()
#type(t)
gc.collect()

In [ ]:



def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)


def to_list(tensor):
    return tensor.detach().cpu().tolist()


def load_data(file_path):
    """
    Used to load CAILExample and CAILFeature objects.
    """
    return pickle.load(gzip.open(file_path, "rb"))


def train(args, train_dataset: TensorDataset, model: nn.Module,
          tokenizer):
    args.train_batch_size = args.batch_size
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    print("***** Running training *****")
    
    logging.info("  Total optimization steps = %d", t_total)

    global_step = 1
    epochs_trained = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch",
    )
    #set_seed(args)

    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(args.device) for t in batch)
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
                "start_labels": batch[3],
                "end_labels": batch[4],
            }
            outputs = model(**inputs)
            loss = outputs[0]
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps
            loss.backward()
            tr_loss += loss.item()
            logging_loss += loss.item()

            if (step + 1) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1

                if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    logging.info(f"Current global step: {global_step}, start evaluating!")
                    logging.info(f"average loss of batch: {logging_loss / args.logging_steps}")
                    logging_loss = 0
                    results = evaluate(args, model, tokenizer, prefix=f"{global_step}-dev",
                                       eval_data_dir=args.eval_data_dir, ground_truth_file=args.ground_truth_file,
                                       multi_span_predict=True)
                    logging.info(f"Evaluation result in dev-set at global step {global_step}: {results}")
                '''
                if args.save_steps > 0 and global_step % args.save_steps == 0:
                    output_dir = os.path.join(args.output_dir, "checkpoint-{}".format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, "module") else model
                    torch.save(model_to_save, os.path.join(output_dir, "checkpoint.bin"))
                    tokenizer.save_pretrained(output_dir)
                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logging.info("Saving model checkpoint to %s", output_dir)
                '''
    return global_step, tr_loss / global_step


def evaluate(args, model, tokenizer, prefix, eval_data_dir: str,
             ground_truth_file: str = None, multi_span_predict=True):
    logging.info(f"Loading data for evaluation from {eval_data_dir}!")
    examples = load_data(join(args.eval_data_dir, "dev_examples.pkl.gz"))
    features = load_data(join(args.eval_data_dir, "dev_features.pkl.gz"))
    dataset = convert_features_to_dataset(features, is_training=False)
    logging.info("Complete Loading!")

    args.eval_batch_size = args.batch_size
    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    logging.info("***** Running evaluation {} *****".format(prefix))
    logging.info("  Num examples = %d", len(dataset))
    logging.info("  Batch size = %d", args.eval_batch_size)

    all_results = []
    start_time = timeit.default_timer()

    for batch in tqdm(eval_dataloader, desc="Evaluating", disable=True):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }
            features_indexes = batch[6]
            outputs = model(**inputs)

        for i, feature_index in enumerate(features_indexes):
            eval_feature = features[feature_index.item()]
            unique_id = int(eval_feature.unique_id)
            output = [to_list(o[i]) for o in outputs]

            start_logits, end_logits = output

            result = CAILResult(
                unique_id=unique_id,
                start_logits=start_logits,
                end_logits=end_logits,
            )
            all_results.append(result)

    evalTime = timeit.default_timer() - start_time
    logging.info("  Evaluation done in total %f secs (%f sec per example)", evalTime, evalTime / len(dataset))

    # Compute predictions
    output_prediction_file = os.path.join(args.output_dir, "predictions_{}.json".format(prefix))
    output_nbest_file = os.path.join(args.output_dir, "nbest_predictions_{}.json".format(prefix))

    compute_predictions(
        examples,
        features,
        all_results,
        args.n_best_size,
        args.max_answer_length,
        args.do_lower_case,
        output_prediction_file,
        output_nbest_file,
        False,
        args.null_score_diff_threshold,
        tokenizer,
        args,
        multi_span_predict=multi_span_predict
    )
    if ground_truth_file is not None:
        evaluator = CJRCEvaluator(ground_truth_file)
        pred_data = CJRCEvaluator.preds_to_dict(output_prediction_file)
        res = evaluator.model_performance(pred_data)
        return res


def main_train_model(args):
    '''
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--model_path",
        default=None,
        type=str,
        required=True,
        help="Path to Electra model."
    )

    parser.add_argument(
        "--train_data_dir",
        default=None,
        required=True,
        type=str,
        help="The directory which contain the generated train-set examples and features file."
    )

    parser.add_argument(
        "--output_dir",
        default=None,
        type=str,
        required=True,
        help="The output directory of checkpoints and predictions."
    )

    parser.add_argument(
        "--eval_data_dir",
        default=None,
        type=str,
        required=True,
        help="The directory which contain the generated dev-set examples and features file."
    )

    parser.add_argument(
        "--ground_truth_file",
        default=None,
        type=str,
        help="The ground truth file of dev-set."
    )

    parser.add_argument(
        "--n_best_size",
        default=20,
        type=int,
        help="The total number of n-best predictions to generate in the nbest_predictions.json output file.",
    )

    parser.add_argument("--device", default="cuda", type=str, help="Whether not to use CUDA when available")
    parser.add_argument("--batch_size", default=8, type=int, help="Batch size per GPU/CPU for training and evaluating.")
    parser.add_argument("--learning_rate", default=3e-5, type=float, help="The initial learning rate for Adam.")
    parser.add_argument("--seed", type=int, default=42, help="random seed for initialization.")
    parser.add_argument(
        "--do_lower_case", action="store_true", help="Set this flag if you are using an uncased model."
    )
    parser.add_argument(
        "--gradient_accumulation_steps",
        type=int,
        default=1,
        help="Number of updates steps to accumulate before performing a backward/update pass.",
    )
    parser.add_argument("--weight_decay", default=0.0, type=float, help="Weight decay if we apply some.")
    parser.add_argument("--adam_epsilon", default=1e-8, type=float, help="Epsilon for Adam optimizer.")
    parser.add_argument("--warmup_steps", default=0, type=int, help="Linear warmup over warmup_steps.")
    parser.add_argument(
        "--num_train_epochs", default=3.0, type=float, help="Total number of training epochs to perform."
    )
    parser.add_argument(
        "--max_steps",
        default=-1,
        type=int,
        help="If > 0: set total number of training steps to perform. Override num_train_epochs.",
    )
    parser.add_argument("--max_grad_norm", default=1.0, type=float, help="Max gradient norm.")
    parser.add_argument("--logging_steps", type=int, default=50, help="Log every X updates steps.")
    parser.add_argument("--save_steps", type=int, default=600, help="Save checkpoint every X updates steps.")

    parser.add_argument(
        "--max_answer_length",
        default=32,
        type=int,
        help="The maximum length of an answer that can be generated. This is needed because the start "
             "and end predictions are not conditioned on one another.",
    )
    parser.add_argument(
        "--null_score_diff_threshold",
        type=float,
        default=0.0,
        help="If null_score - best_non_null is greater than the threshold predict null.",
    )
    parser.add_argument(
        "--max_seq_length",
        default=512,
        type=int,
        help="The maximum total input sequence length after WordPiece tokenization. "
             "Longer will be truncated, and shorter will be padded."
    )
    parser.add_argument(
        "--max_query_length",
        default=64,
        type=int,
        help="The maximum number of tokens for the question. Questions longer will be truncated to the length."
    )
    parser.add_argument(
        "--doc_stride",
        default=128,
        type=int,
        help="When splitting up a long document into chunks, how much stride to take between chunks."
    )

    parser.add_argument(
        "--multi_span_threshold",
        type=float,
        default=0.8,
        help="Span which score is bigger than (max_span_score * multi_span_threshold) will also be output!"
    )
    '''
    #args = parser.parse_args()
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
    )

    logging.info("All input parameters:")
    #print(json.dumps(vars(args), sort_keys=False, indent=2))

    #set_seed(args)

    if not os.path.exists(args.output_dir):
        os.makedirs(args.output_dir)
    
    config = AutoConfig.from_pretrained(args_train.model_path)
    tokenizer = AutoTokenizer.from_pretrained(args_train.model_path)
    pretrain_model = AutoModel.from_pretrained(args_train.model_path, config=config)
    model = MultiSpanQA(pretrain_model)

    model.to(device=args_train.device)

    logging.info("Loading pre-processed examples and features!")
    train_examples = load_data(args_train.train_data_dir + "/train_examples.pkl.gz")
    train_features = load_data(args_train.train_data_dir + "/train_features.pkl.gz")
    print("Complete loading!")

    logging.info("Converting features to pytorch Dataset!")
    train_dataset = convert_features_to_dataset(train_features, is_training=True)
    logging.info("Complete converting!")

    global_step, tr_loss = train(args_train, train_dataset, model, tokenizer)
    logging.info("global_step = %s, average loss = %s", global_step, tr_loss)
    logging.info("Training Complete!")

    logging.info("Saving model checkpoint to %s", args.output_dir)
    model_to_save = model.module if hasattr(model, "module") else model
    torch.save(model_to_save, os.path.join(args_train.output_dir, "checkpoint.bin"))
    torch.save(args_train, os.path.join(args_train.output_dir, "training_args.bin"))
    tokenizer.save_pretrained(args_train.output_dir)
    results = evaluate(args_train, model, tokenizer, prefix=f"final-eval",
                       eval_data_dir=args.eval_data_dir, ground_truth_file=args.ground_truth_file,
                       multi_span_predict=True)
    logging.info(f"Final evaluate results on dev-set: {results}")


#if __name__ == "__main__":
main_train_model(args_train)

In [ ]:
try:
    print(os.listdir( './pocessed_data/'))
except:
    print('error last step')